## Скачиваю данные с сайта Роспотребнадзора:

In [20]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]
    
urls = []
for page in tqdm(range(1, 500), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))

Pages:   0%|          | 0/499 [00:00<?, ?it/s]

In [3]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')

Pages:   0%|          | 0/2495 [00:00<?, ?it/s]

In [4]:
## посмотрим, что скачалось
N = 182
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Три недели перебои работы телевизора через интернет приставку, служба поддержки отвечает с вами свяжутся,  на протяжении трёх недель, как решить вопрос

A: Добрый день! ВАм необходимо обратиться с претензией в обслуживающую организацию. В случае не урегулирования вопроса в Роскомнадзор


In [5]:
question

['На перекрестке улиц Генкиной и Ижорской в Нижнем Новгороде уже который день ведутся ремонтные работы в будние дни по ночам. Работы ведутся крайне шумно и мешают спать многочисленным жильцам в прилегающих в максимальной близости многоэтажных домах. Рабочие дробят асфальт, бьют молотом по металлу и производят другие крайне шумные работы. Машины стоят с включёнными двигателями.',
 'Добрый день! Просьба принять жалобу на точку питания шаурма "на труда", расположенную по адресу: Челябинская область, г. Челябинск,  \u200bПроспект Победы, 396 ст1\r\n30.07.2023 сделали заказ \r\n31.07.2023 человека забрали с острым отравлением в инфекционное отделение \r\n02.08.2023 уже с отравлением лежу я \r\nтакже просмотрели отзывы от этих чисел и люди массово жалуются на отравления!\r\nпросьба принять меры к этому заведению.',
 'Заказывали коллективом шаурму заведение г.Челябинска «на Труда» по адресу: Труда 148. На следующей день все с диареей! Зашла в отзывы этого кафе, у всех кто заказывал в эти дни 

In [6]:
type(question)

list

In [7]:
len(question)

2495

In [21]:
df = pd.DataFrame(question, columns=['question'])

In [22]:
df

,question
0,На перекрестке улиц Генкиной и Ижорской в Нижн...
1,Добрый день! Просьба принять жалобу на точку п...
2,Заказывали коллективом шаурму заведение г.Челя...
3,Я оплатил услуги связи в том числе за 25гигаба...
4,"В магазине ""Валентинка"" находившийся по адресу..."
...,...
2490,\r\nДобрый день!\r\nПрошу рассмотреть мою жало...
2491,"По адресу г. ОВОСИБИРСК, УЛ. 1905 ГОДА, 85 Ото..."
2492,"По адресу г. ОВОСИБИРСК, УЛ. 1905 ГОДА, 85 Ото..."
2493,"Меня зовут Квитка И.А., \r\n11.10.2022 во двор..."


In [23]:
df.to_csv('question.csv', index=False)

In [24]:
df.to_csv('question.csv', index=False)

In [25]:
df = pd.read_csv('question.csv')

df

,question
0,На перекрестке улиц Генкиной и Ижорской в Нижн...
1,Добрый день! Просьба принять жалобу на точку п...
2,Заказывали коллективом шаурму заведение г.Челя...
3,Я оплатил услуги связи в том числе за 25гигаба...
4,"В магазине ""Валентинка"" находившийся по адресу..."
...,...
2490,\r\nДобрый день!\r\nПрошу рассмотреть мою жало...
2491,"По адресу г. ОВОСИБИРСК, УЛ. 1905 ГОДА, 85 Ото..."
2492,"По адресу г. ОВОСИБИРСК, УЛ. 1905 ГОДА, 85 Ото..."
2493,"Меня зовут Квитка И.А., \r\n11.10.2022 во двор..."
